In [1]:
#!conda install ipywidgets

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
import zipfile

In [3]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [4]:
from datetime import datetime

In [5]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [6]:
class DataSet:
    def __init__(self):
        self.df = self.get_dataframe()
        self.features = self.df.columns
        self.unique_values = self.get_unique_values()
        
    def get_dataframe(self):
        data_url = 'http://sbrfdata.ru/opendata.zip'
        zip_data = requests.get(data_url).content
        with open('opendata.zip', 'wb') as file:
            file.write(zip_data)
        with zipfile.ZipFile('opendata.zip') as zip_file:
            zip_file.extractall()
        return pd.read_csv('opendata.csv', encoding='cp1251', sep=',', parse_dates=[2], 
                           date_parser=lambda date_string : self._date_from_string(date_string))
    
    def get_unique_values(self):
        unique_values = {}
        for feature in self.features:
            unique_values[feature] = self.df[feature].unique()
        return unique_values
    def _date_from_string(self, date_string):
        parsed = date_string.split('-')
        parsed = [int(val) for val in parsed]
        return pd.to_datetime(date_string)
        return datetime(parsed[0], parsed[1], parsed[2])

In [7]:
ds = DataSet()

### Вывод данных в табличном виде (поддерживает фильтрацию и сортировку):

In [8]:
@interact
def show_filtered(param=ds.unique_values['name'],
                  region=ds.unique_values['region'],
                  date_from=widgets.DatePicker(value=pd.to_datetime('2015-01-01')),
                  date_till=widgets.DatePicker(value=pd.to_datetime('2019-01-01')),
                  show_values=(1, 19),
                  sort_by=widgets.RadioButtons(options=['name', 'region', 'date', 'value'], value='value'),
                  direction=widgets.RadioButtons(options=['^', 'v'])):
    return ds.df[(ds.df['name'] == param) &
                 (ds.df['region'] == region) &
                 (ds.df['date'] < pd.to_datetime(date_till)) &
                 (ds.df['date'] > pd.to_datetime(date_from))].sort_values(by=sort_by,
                                                                          ascending= direction=='^').head(show_values)

interactive(children=(Dropdown(description='param', options=('Количество заявок на потребительские кредиты', '…

### Вывод данных в графическом виде (динамика показателей):

In [9]:
@interact
def show_plot(param=ds.unique_values['name'],
                  region=ds.unique_values['region'],
                  date_from=widgets.DatePicker(value=pd.to_datetime('2015-01-01')),
                  date_till=widgets.DatePicker(value=pd.to_datetime('2019-01-01'))):
    df = ds.df[(ds.df['name'] == param) &
                 (ds.df['region'] == region) &
                 (ds.df['date'] < pd.to_datetime(date_till)) &
                 (ds.df['date'] > pd.to_datetime(date_from))].sort_values(by='date')
    plt.figure(figsize=(10, 5))
    return plt.plot(df['date'],
                 df['value'],
                    color='green',
                    linewidth=5)

interactive(children=(Dropdown(description='param', options=('Количество заявок на потребительские кредиты', '…